In [7]:
import os
import numpy as np

from plotly.graph_objects import Figure
from plotly.graph_objects import Scatter

from  sxo.util.runtime.timeseries import RedisTs

In [8]:
os.environ["REDIS_PASS"] = "boss"


In [9]:
ts_bid = RedisTs('ts_bid_37374')
ts_ask = RedisTs('ts_ask_37374')
bid = ts_bid.get_range(name='bid')
ask = ts_ask.get_range(name='ask')

In [10]:
bid_ask = bid.join(ask, how='inner').reset_index()
bid_ask['t'] = bid_ask['t'].values.astype('datetime64[ms]')

In [11]:
df_plt = bid_ask.copy() #[bid_ask.t > np.datetime64('2023-06-06 20:20:00')]

fig = Figure()
fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['ask'], name='ask', mode="markers", marker={'size':4}))
fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['bid'], name='bid', mode="markers", marker={'size':4}))
fig.show()